In [1]:
import cv2
import numpy as np
import imutils
import csv
import sqlite3

# Define constants and paths
MIN_CONF = 0.3
NMS_THRESH = 0.3
MIN_DISTANCE = 50
MODEL_PATH = "D:\\Video_analytic_assignment\\Project\\coco.names"
weightsPath = "D:\\Video_analytic_assignment\\Project\\yolov3.weights"
configPath = "D:\\Video_analytic_assignment\\Project\\yolov3.cfg"

# Load the YOLO model
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
labels = open(MODEL_PATH).read().strip().split("\n")
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

# Define function to detect people
def detect_people(frame, net, ln, personIdx=0):
    # grab the dimensions of the frame
    (H, W) = frame.shape[:2]

    # initialize the list of results
    results = []

    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (832, 832), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    # initialize our lists of detected bounding boxes, centroids, and
    # confidences, respectively
    boxes = []
    centroids = []
    confidences = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter detections by (1) ensuring that the object
            # detected was a person and (2) that the minimum
            # confidence is met
            if classID == personIdx and confidence > MIN_CONF:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # centroids, and confidences
                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # update our results list to consist of the person
            # prediction probability, bounding box coordinates,
            # and the centroid
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)

    return results

# Function to generate density map
def generate_density_map(frame, results):
    density_map = np.zeros(frame.shape[:2], dtype=np.float32)

    # Generate density map based on detected people
    for (_, _, centroid) in results:
        (cX, cY) = centroid
        density_map[cY, cX] += 1

    # Apply Gaussian blur to smooth density map
    density_map = cv2.GaussianBlur(density_map, (21, 21), 0)

    # Normalize density map
    density_map = cv2.normalize(density_map, None, 0, 255, cv2.NORM_MINMAX)
    density_map = np.uint8(density_map)

    return density_map

# Function to estimate number of people based on density
def estimate_people_count(density_map, threshold):
    # Threshold the density map to count the number of people
    _, binary_map = cv2.threshold(density_map, threshold, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    people_count = len(contours)

    return people_count

# Function to overlay heatmap on frame
def overlay_heatmap(frame, density_map):
    # Convert density map to RGB format
    heatmap = cv2.applyColorMap(density_map, cv2.COLORMAP_JET)

    # Overlay heatmap on frame
    frame_with_heatmap = cv2.addWeighted(frame, 0.6, heatmap, 0.4, 0)

    return frame_with_heatmap

# Initialize video capture
vs = cv2.VideoCapture("D:\\Video_analytic_assignment\\Project\\new_1.mp4")

# Connect to the SQLite database
conn = sqlite3.connect("D:\\Video_analytic_assignment\\Crowd_counter_project\\output_path\\video_data.db")
cur = conn.cursor()

# Create a table to store the frame number and people count
cur.execute('''CREATE TABLE IF NOT EXISTS video_data
               (frame_number INTEGER PRIMARY KEY, people_count INTEGER)''')

# Loop over video frames
frame_number = 0
while True:
    # Read the next frame
    (grabbed, frame) = vs.read()
    frame_number += 1

    # If the frame was not grabbed, exit loop
    if not grabbed:
        break

    # Resize the frame and detect people in it
    frame = imutils.resize(frame, width=700)
    results = detect_people(frame, net, ln, personIdx=labels.index("person"))

    # Print the length of results and frame number for debugging
    print("Frame Number:", frame_number)
    print("Number of People Detected:", len(results))

    # Check if results is None
    if results is None:
        print("No people detected in the frame.")
        continue

    # Generate density map based on detected people
    density_map = generate_density_map(frame, results)
    # Estimate number of people based on density
    threshold = 100  # Adjust this threshold value as needed
    people_count = estimate_people_count(density_map, threshold)

    # Overlay heatmap on frame
    frame_with_heatmap = overlay_heatmap(frame, density_map)

    # Display the frame with heatmap and people count
    cv2.putText(frame_with_heatmap, f"People Count: {people_count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Frame with Heatmap and People estimate", frame_with_heatmap)

    # Store data in the SQLite database
    # cur.execute("INSERT INTO video_data (frame_number, people_count) VALUES (?, ?)", (frame_number, people_count))
    conn.commit()

    key = cv2.waitKey(1) & 0xFF

    # If 'q' key pressed, break from loop
    if key == ord("q"):
        break

# Release video capture and close windows
vs.release()
cv2.destroyAllWindows()

# Close the SQLite connection
conn.close()


Frame Number: 1
Number of People Detected: 57
Frame Number: 2
Number of People Detected: 56
Frame Number: 3
Number of People Detected: 54
Frame Number: 4
Number of People Detected: 49
Frame Number: 5
Number of People Detected: 49
Frame Number: 6
Number of People Detected: 54
Frame Number: 7
Number of People Detected: 62
Frame Number: 8
Number of People Detected: 57
Frame Number: 9
Number of People Detected: 57
Frame Number: 10
Number of People Detected: 55
Frame Number: 11
Number of People Detected: 51
Frame Number: 12
Number of People Detected: 55
Frame Number: 13
Number of People Detected: 53
Frame Number: 14
Number of People Detected: 52
Frame Number: 15
Number of People Detected: 52
Frame Number: 16
Number of People Detected: 49
Frame Number: 17
Number of People Detected: 45
Frame Number: 18
Number of People Detected: 46
Frame Number: 19
Number of People Detected: 43
Frame Number: 20
Number of People Detected: 48
Frame Number: 21
Number of People Detected: 52
Frame Number: 22
Numbe

In [2]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("D:\\Video_analytic_assignment\\Crowd_counter_project\\output_path\\video_data.db")
cur = conn.cursor()

# Execute a simple query to fetch all data from the table
# cur.execute("SELECT * FROM video_data")
cur.execute("SELECT * FROM video_data LIMIT 10")
rows = cur.fetchall()

# Print the fetched data
for row in rows:
    print(row)

# Close the connection
conn.close()


(1, 55)
(2, 56)
(3, 54)
(4, 49)
(5, 49)
(6, 54)
(7, 59)
(8, 56)
(9, 56)
(10, 54)
